In [1]:
import pandas as pd
import re, subprocess, pyodbc , os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [8]:
lst = ['EinsteinPublicIP']
engine = create_engine(connstr) 
for t in lst:    
    query = f""" \
        SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{t}' 
    """ 
    conn = engine.connect()
    df = pd.read_sql(query,con=conn)     
    conn.close()
    df['Class'] = df['TABLE_NAME'].apply(lambda s: s.replace('fsma_',''))
    df['Class'] = df['Class'].apply(lambda s:  re.sub('s$','',s))
    df['DataType'] = df['DATA_TYPE'].apply(lambda s: re.sub('(nvarchar|nchar|bit|varchar)','string',s)) 
    df['DataType'] = df['DataType'].apply(lambda s: re.sub('(smalldatetime|datetime)','DateTime',s))  
    df[['Class', 'DataType','COLUMN_NAME','TABLE_NAME']] 
    template = """
using System;
using System.Collections.Generic; 
namespace CyberScope.Tests
{
    [CSEntityMeta("{TABLE_NAME}")]
    public class {0}
    {
        public {0}(){ 
        
        } {1}
    } 
}
    """ 
    template = template.replace('{0}', df.iloc[0]['Class'])
    template = template.replace('{TABLE_NAME}', df.iloc[0]['TABLE_NAME']) 
    c=''
    for i,r in df.iterrows():
        c = c + '\n\t\t[CSEntityPropertyMeta("{1}")]'.replace('{1}',r.COLUMN_NAME)
        c = c + '\n\t\tpublic {0} {1} { get; set; }'.replace('{0}',r.DataType).replace('{1}',r.COLUMN_NAME)
    template = template.replace('{1}', c) 
print(template) 
conn.close() 


using System;
using System.Collections.Generic; 
namespace CyberScope.Tests
{
    [CSEntityMeta("EinsteinPublicIP")]
    public class EinsteinPublicIP
    {
        public EinsteinPublicIP(){ 
        
        } 
		[CSEntityPropertyMeta("PK_EinsteinPublicIP")]
		public int PK_EinsteinPublicIP { get; set; }
		[CSEntityPropertyMeta("PK_OrgSubmission")]
		public int PK_OrgSubmission { get; set; }
		[CSEntityPropertyMeta("IPAddress")]
		public string IPAddress { get; set; }
		[CSEntityPropertyMeta("Name")]
		public string Name { get; set; }
		[CSEntityPropertyMeta("ISP")]
		public string ISP { get; set; }
		[CSEntityPropertyMeta("OtherISP")]
		public string OtherISP { get; set; }
		[CSEntityPropertyMeta("ExternalOrg")]
		public string ExternalOrg { get; set; }
		[CSEntityPropertyMeta("Visible")]
		public int Visible { get; set; }
		[CSEntityPropertyMeta("UserId")]
		public int UserId { get; set; }
		[CSEntityPropertyMeta("isActive")]
		public string isActive { get; set; }
		[CSEntityPrope

In [2]:
lst = ['fsma_Answers']
engine = create_engine(connstr) 
for t in lst:    
    query = f""" \
        SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{t}' 
    """ 
    conn = engine.connect()
    df = pd.read_sql(query,con=conn)     
    conn.close()
    df['Class'] = df['TABLE_NAME'].apply(lambda s: s.replace('fsma_',''))
    df['Class'] = df['Class'].apply(lambda s:  re.sub('s$','',s))
    df['DataType'] = df['DATA_TYPE'].apply(lambda s: re.sub('(nvarchar|nchar|bit|varchar)','string',s)) 
    df['DataType'] = df['DataType'].apply(lambda s: re.sub('(smalldatetime)','DateTime',s)) 
  
    df[['Class', 'DataType','COLUMN_NAME','TABLE_NAME']] 
    template = """
using System;
using System.Collections.Generic; 
namespace CyberScope.Tests
{
    [EntityMeta("{TABLE_NAME}")]
    public class {0}
    {
        public {0}(){ 
        
        } {1}
    } 
}
    """ 
    template = template.replace('{0}', df.iloc[0]['Class'])
    template = template.replace('{TABLE_NAME}', df.iloc[0]['TABLE_NAME']) 
    c=''
    for i,r in df.iterrows():
        c = c + '\n\t\tpublic {0} {1} { get; set; }'.replace('{0}',r.DataType).replace('{1}',r.COLUMN_NAME)
    template = template.replace('{1}', c) 
    #tmps = tmps + template
    fname = r'D:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope.Tests\\Models\\' +df.iloc[0]['Class']+'.cs' 
with open(fname, 'w') as f:
    f.write(template);
conn.close()    

In [95]:
def getvar( s ):
    l = s.split('-') 
    return f'{l[2]}_{l[1]}_{l[0]}'

In [98]:
engine = create_engine(connstr)     
query = f""" \
    SELECT TOP 25  PK_FORM FROM fsma_FormMaster ORDER BY Report_Year DESC
""" 
c=''
conn = engine.connect()
dff = pd.read_sql(query,con=conn)     
conn.close()
dff['VAR'] = dff['PK_FORM'].apply(getvar)
for i,r in dff.iterrows(): 
    c = c + f'public static string {r.VAR} = "{r.PK_FORM}";' + '\n'
#fname='D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Models\'    
#fname=fname+'PKForm.cs' 
print (c)
#with open(fname, 'w') as f:
#    f.write(template);


public static string ED2101A_A_2021 = "2021-A-ED2101A";
public static string ED2101N_A_2021 = "2021-A-ED2101N";
public static string EINSTEIN_A_2021 = "2021-A-EINSTEIN";
public static string HVA_A_2021 = "2021-A-HVA";
public static string IG_A_2021 = "2021-A-IG";
public static string SAOP_A_2021 = "2021-A-SAOP";
public static string CIO_Q1_2021 = "2021-Q1-CIO";
public static string RMA_Q1_2021 = "2021-Q1-RMA";
public static string BODVDP_Q2_2021 = "2021-Q2-BODVDP";
public static string CIO_Q2_2021 = "2021-Q2-CIO";
public static string CYBEREO_Q2_2021 = "2021-Q2-CYBEREO";
public static string RMA_Q2_2021 = "2021-Q2-RMA";
public static string CIO_Q3_2021 = "2021-Q3-CIO";
public static string RMA_Q3_2021 = "2021-Q3-RMA";
public static string CIO_Q4_2021 = "2021-Q4-CIO";
public static string RMA_Q4_2021 = "2021-Q4-RMA";
public static string AAPS_A_2020 = "2020-A-AAPS";
public static string HVA_A_2020 = "2020-A-HVA";
public static string HVAPOAM_A_2020 = "2020-A-HVAPOAM";
public static stri

In [ ]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w[:15] for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper().strip()
    #t = re.sub('\s', '_', t) 
    t = re.sub('[AEIOU\s]', '', t)  
    return t
encode(t)

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
	SELECT 
	  vw_MetricsCompositeKeys.QGROUP
	, vw_MetricsCompositeKeys.QPK
	, vw_MetricsCompositeKeys.QTEXT
	, vw_MetricsCompositeKeys.IdText
	, vw_MetricsCompositeKeys.GroupName
	, vw_MetricsCompositeKeys.GroupDesc
	, fsma_ReportingCycles.PK_ReportingCycle
	, fsma_ReportingCycles.Description
	, fsma_FormMaster.Report_Year as Year
	, fsma_FormMaster.Form_Description 
	, fsma_FormMaster.IntervalCode 
	, fsma_FormMaster.Period 
    , fsma_FormMaster.PK_Form
	FROM vw_MetricsCompositeKeys 
	INNER JOIN fsma_FormMaster ON vw_MetricsCompositeKeys.PK_Form=fsma_FormMaster.PK_Form
	INNER JOIN fsma_ReportingCycles ON fsma_FormMaster.FK_ReportingCycle=fsma_ReportingCycles.PK_ReportingCycle
 
	WHERE vw_MetricsCompositeKeys.PK_Form LIKE '%CIO'  
	AND Year > 2010

""" 
df = pd.read_sql(query,con=conn)     
conn.close()